In [1]:
import CharacterizingAffineCSemigroup
from CharacterizingAffineCSemigroup import *

In [2]:
gen0 = [[4,0],[3,0],[5,0],[5,5],[2,2],[3,3],[4,3],[5,3],[5,1],[6,1],[7,1],[5,4]]

In [3]:
IsCsemigroup(gen0)

True

In [4]:
C0 = Cone(cone=gen0)
rayos0 = C0.ExtremeRays()
rayos0

[[1, 0], [1, 1]]

In [5]:
# Calculamos la multiplicidad en cada rayo.
# INPUT:
#   - ray: rayo del cono.
#   - smg: sistema minimal de generadores.
# OUTPUT:
#   - Elemento minimal en el rayo.
def MultiplicityAxis(ray,smg):
    multiplicity = []
    for x in smg:
        aux = BelongAxis(x,ray)
        if aux != 0:
            multiplicity.append([aux,x])
    return sorted(multiplicity)[0][1]

In [6]:
MultiplicityAxis([1,0],gen0)

[3, 0]

In [7]:
# Calculamos el diamante entero de las multiplicidades.
# INPUT:
#   - cone: rayos del cono.
#   - smg: sistema minimal de generadores.
# OUTPUT:
#   - diamante entero de las multiplicidades
def DiamondMultiplicity(cone,smg):
    extremos = []
    for ray in cone:
        extremos.append(MultiplicityAxis(ray,smg))
    diamante = Diamond(extremos)
    hull = ConvexHull(diamante)
    eq = [list(x) for x in hull.equations]
    cotas = Cube(diamante)
    return IntegerDiamond(eq,cotas)

In [8]:
d=DiamondMultiplicity([[1,0],[1,1]],gen0)

In [9]:
# Ecuaciones de un rayo.
eq = EqRay([1,0],[2,1])

In [10]:
# Borramos elementos repetidos.
# INPUT:
#   - v: vector de entrada.
# OUTPUT:
#   - vector v sin elementos de entrada.
def DeleteDuplicates(v):
    w = []
    for i in range(len(v)):
        if v[i] not in w:
            w.append(v[i])
    return w

In [11]:
DeleteDuplicates(AffineTerm(eq,d))

[[1], [2]]

In [12]:
# Creamos una función que calcula la uno norma.
# INPUT:
#   - v: un vector.
# OUTPUT:
#   - Norma-1 del vector v.
def NormOne(v):
    suma = 0
    for x in v:
        suma = suma + x
    return suma

In [13]:
NormOne([3,4,6,8,0])

21

In [14]:
# Calculamos el punto de menor norma del diamante para una recta afín.
# INPUT:
#   - diamond: diamante entero.
#   - afin: término afin de la recta.
#   - eqray: ecuaciones de la recta.
# OUTPUT:
#   - Punto de menor norma del diamante que pasa por la recta afín.
def MinimumPointAffineDiamond(diamond,afin,eqray):
    numeq = len(eqray)
    aux2 = []
    for d in diamond:
        aux = MultiplyMatrix(eqray,[[x] for x in d])
        if aux == [afin]:
            aux2.append([NormOne(d),d])
    return(sorted(aux2)[0][1])

In [15]:
MinimumPointAffineDiamond(d,[1],eq)

[1, 1]

In [16]:
# Calculamos los elementos de norma mínima por cada recta afín.
# INPUT:
#   - vdir: vector director de la recta.
#   - seed: punto para empezar a mirar la recta.
#   - smgen: sistema minimal de generadores.
# OUTPUT:
#   - Norma mínima de los elementos del semigrupo de la recta. 
def ComputeMinimumNormInSemigroupLine(vdir,seed,smgen):
    v = seed
    afseg = AffineSemigroup(smgen, "generators")
    while True:
        if afseg.belongs(v):
            return NormOne(v)
        v = [v[i] + seed[i] for i in range(len(vdir))]

In [17]:
ComputeMinimumNormInSemigroupLine([1,0],[1,1],gen0)

4

In [32]:
# Calcula un valor en el rayo y en el semigrupo con norma mayor que una dada.
# INPUT:
#   - n: cota de la norma.
#   - ray: rayo del cono.
#   - gen: generadores del semigrupo.
# OUTPUT:
#   - Valor en el rayo que está en el semigrupo y tiene norma mayor que n.
def ComputeBoundRay(n,ray,gen):
    afseg = AffineSemigroup(gen, "generators")
    v = ray
    i = 1
    while True:
        v = [i*x for x in ray]
        print(v)
        print(afseg.belongs(v))
        if NormOne(v) > n and afseg.belongs(v):
            return v
        i = i+1

In [38]:
sgaf = AffineSemigroup(gen0, "generators")

In [42]:
AffineSemigroup(gen0, "generators").getMSG()

[[5, 1], [5, 4], [3, 3], [7, 1], [3, 0], [6, 1], [5, 0], [2, 2], [4, 3], [5, 3
], [4, 0]]

In [44]:
sgaf.belongs([9,0])

False

In [40]:
sgaf.getExpressions([8,0])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]]

In [37]:
gen0

[[4, 0], [3, 0], [5, 0], [5, 5], [2, 2], [3, 3], [4, 3], [5, 3], [5, 1], [6, 1
], [7, 1], [5, 4]]

In [33]:
ComputeBoundRay(7,[1,0],gen0)

[1, 0]
False
[2, 0]
False
[3, 0]
True
[4, 0]
True
[5, 0]
True
[6, 0]
False
[7, 0]
False
[8, 0]
False
[9, 0]
False
[10, 0]
False
[11, 0]
False
[12, 0]
False
[13, 0]
False
[14, 0]
False
[15, 0]
False
[16, 0]
False
[17, 0]
False
[18, 0]
False
[19, 0]
False
[20, 0]
False
[21, 0]
False
[22, 0]
False
[23, 0]
False
[24, 0]
False
[25, 0]
False
[26, 0]
False
[27, 0]
False
[28, 0]
False
[29, 0]
False
[30, 0]
False
[31, 0]
False
[32, 0]
False
[33, 0]
False
[34, 0]
False
[35, 0]
False
[36, 0]
False
[37, 0]
False
[38, 0]
False
[39, 0]
False
[40, 0]
False
[41, 0]
False
[42, 0]
False
[43, 0]
False
[44, 0]
False
[45, 0]
False
[46, 0]
False
[47, 0]
False
[48, 0]
False
[49, 0]
False
[50, 0]
False
[51, 0]
False
[52, 0]
False
[53, 0]
False
[54, 0]
False
[55, 0]
False
[56, 0]
False
[57, 0]
False
[58, 0]
False
[59, 0]
False
[60, 0]
False
[61, 0]
False
[62, 0]
False
[63, 0]
False
[64, 0]
False
[65, 0]
False
[66, 0]
False
[67, 0]
False
[68, 0]
False
[69, 0]
False
[70, 0]
False
[71, 0]
False
[72, 0]
False
[73,

False
[868, 0]
False
[869, 0]
False
[870, 0]
False
[871, 0]
False
[872, 0]
False
[873, 0]
False
[874, 0]
False
[875, 0]
False
[876, 0]
False
[877, 0]
False
[878, 0]
False
[879, 0]
False
[880, 0]
False
[881, 0]
False
[882, 0]
False
[883, 0]
False
[884, 0]
False
[885, 0]
False
[886, 0]
False
[887, 0]
False
[888, 0]
False
[889, 0]
False
[890, 0]
False
[891, 0]
False
[892, 0]
False
[893, 0]
False
[894, 0]
False
[895, 0]
False
[896, 0]
False
[897, 0]
False
[898, 0]
False
[899, 0]
False
[900, 0]
False
[901, 0]
False
[902, 0]
False
[903, 0]
False
[904, 0]
False
[905, 0]
False
[906, 0]
False
[907, 0]
False
[908, 0]
False
[909, 0]
False
[910, 0]
False
[911, 0]
False
[912, 0]
False
[913, 0]
False
[914, 0]
False
[915, 0]
False
[916, 0]
False
[917, 0]
False
[918, 0]
False
[919, 0]
False
[920, 0]
False
[921, 0]
False
[922, 0]
False
[923, 0]
False
[924, 0]
False
[925, 0]
False
[926, 0]
False
[927, 0]
False
[928, 0]
False
[929, 0]
False
[930, 0]
False
[931, 0]
False
[932, 0]
False
[933, 0]
False
[934

[1867, 0]
False
[1868, 0]
False
[1869, 0]
False
[1870, 0]
False
[1871, 0]
False
[1872, 0]
False
[1873, 0]
False
[1874, 0]
False
[1875, 0]
False
[1876, 0]
False
[1877, 0]
False
[1878, 0]
False
[1879, 0]
False
[1880, 0]
False
[1881, 0]
False
[1882, 0]
False
[1883, 0]
False
[1884, 0]
False
[1885, 0]
False
[1886, 0]
False
[1887, 0]
False
[1888, 0]
False
[1889, 0]
False
[1890, 0]
False
[1891, 0]
False
[1892, 0]
False
[1893, 0]
False
[1894, 0]
False
[1895, 0]
False
[1896, 0]
False
[1897, 0]
False
[1898, 0]
False
[1899, 0]
False
[1900, 0]
False
[1901, 0]
False
[1902, 0]
False
[1903, 0]
False
[1904, 0]
False
[1905, 0]
False
[1906, 0]
False
[1907, 0]
False
[1908, 0]
False
[1909, 0]
False
[1910, 0]
False
[1911, 0]
False
[1912, 0]
False
[1913, 0]
False
[1914, 0]
False
[1915, 0]
False
[1916, 0]
False
[1917, 0]
False
[1918, 0]
False
[1919, 0]
False
[1920, 0]
False
[1921, 0]
False
[1922, 0]
False
[1923, 0]
False
[1924, 0]
False
[1925, 0]
False
[1926, 0]
False
[1927, 0]
False
[1928, 0]
False
[1929, 0

[2867, 0]
False
[2868, 0]
False
[2869, 0]
False
[2870, 0]
False
[2871, 0]
False
[2872, 0]
False
[2873, 0]
False
[2874, 0]
False
[2875, 0]
False
[2876, 0]
False
[2877, 0]
False
[2878, 0]
False
[2879, 0]
False
[2880, 0]
False
[2881, 0]
False
[2882, 0]
False
[2883, 0]
False
[2884, 0]
False
[2885, 0]
False
[2886, 0]
False
[2887, 0]
False
[2888, 0]
False
[2889, 0]
False
[2890, 0]
False
[2891, 0]
False
[2892, 0]
False
[2893, 0]
False
[2894, 0]
False
[2895, 0]
False
[2896, 0]
False
[2897, 0]
False
[2898, 0]
False
[2899, 0]
False
[2900, 0]
False
[2901, 0]
False
[2902, 0]
False
[2903, 0]
False
[2904, 0]
False
[2905, 0]
False
[2906, 0]
False
[2907, 0]
False
[2908, 0]
False
[2909, 0]
False
[2910, 0]
False
[2911, 0]
False
[2912, 0]
False
[2913, 0]
False
[2914, 0]
False
[2915, 0]
False
[2916, 0]
False
[2917, 0]
False
[2918, 0]
False
[2919, 0]
False
[2920, 0]
False
[2921, 0]
False
[2922, 0]
False
[2923, 0]
False
[2924, 0]
False
[2925, 0]
False
[2926, 0]
False
[2927, 0]
False
[2928, 0]
False
[2929, 0

False
[3867, 0]
False
[3868, 0]
False
[3869, 0]
False
[3870, 0]
False
[3871, 0]
False
[3872, 0]
False
[3873, 0]
False
[3874, 0]
False
[3875, 0]
False
[3876, 0]
False
[3877, 0]
False
[3878, 0]
False
[3879, 0]
False
[3880, 0]
False
[3881, 0]
False
[3882, 0]
False
[3883, 0]
False
[3884, 0]
False
[3885, 0]
False
[3886, 0]
False
[3887, 0]
False
[3888, 0]
False
[3889, 0]
False
[3890, 0]
False
[3891, 0]
False
[3892, 0]
False
[3893, 0]
False
[3894, 0]
False
[3895, 0]
False
[3896, 0]
False
[3897, 0]
False
[3898, 0]
False
[3899, 0]
False
[3900, 0]
False
[3901, 0]
False
[3902, 0]
False
[3903, 0]
False
[3904, 0]
False
[3905, 0]
False
[3906, 0]
False
[3907, 0]
False
[3908, 0]
False
[3909, 0]
False
[3910, 0]
False
[3911, 0]
False
[3912, 0]
False
[3913, 0]
False
[3914, 0]
False
[3915, 0]
False
[3916, 0]
False
[3917, 0]
False
[3918, 0]
False
[3919, 0]
False
[3920, 0]
False
[3921, 0]
False
[3922, 0]
False
[3923, 0]
False
[3924, 0]
False
[3925, 0]
False
[3926, 0]
False
[3927, 0]
False
[3928, 0]
False
[3

...

[25525, 0]
False
[25526, 0]
False
[25527, 0]
False
[25528, 0]
False
[25529, 0]
False
[25530, 0]
False
[25531, 0]
False
[25532, 0]
False
[25533, 0]
False
[25534, 0]
False
[25535, 0]
False
[25536, 0]
False
[25537, 0]
False
[25538, 0]
False
[25539, 0]
False
[25540, 0]
False
[25541, 0]
False
[25542, 0]
False
[25543, 0]
False
[25544, 0]
False
[25545, 0]
False
[25546, 0]
False
[25547, 0]
False
[25548, 0]
False
[25549, 0]
False
[25550, 0]
False
[25551, 0]
False
[25552, 0]
False
[25553, 0]
False
[25554, 0]
False
[25555, 0]
False
[25556, 0]
False
[25557, 0]
False
[25558, 0]
False
[25559, 0]
False
[25560, 0]
False
[25561, 0]
False
[25562, 0]
False
[25563, 0]
False
[25564, 0]
False
[25565, 0]
False
[25566, 0]
False
[25567, 0]
False
[25568, 0]
False
[25569, 0]
False
[25570, 0]
False
[25571, 0]
False
[25572, 0]
False
[25573, 0]
False
[25574, 0]
False
[25575, 0]
False
[25576, 0]
False
[25577, 0]
False
[25578, 0]
False
[25579, 0]
False
[25580, 0]
False
[25581, 0]
False
[25582, 0]
False
[25583, 0]
Fal

False
[26361, 0]
False
[26362, 0]
False
[26363, 0]
False
[26364, 0]
False
[26365, 0]
False
[26366, 0]
False
[26367, 0]
False
[26368, 0]
False
[26369, 0]
False
[26370, 0]
False
[26371, 0]
False
[26372, 0]
False
[26373, 0]
False
[26374, 0]
False
[26375, 0]
False
[26376, 0]
False
[26377, 0]
False
[26378, 0]
False
[26379, 0]
False
[26380, 0]
False
[26381, 0]
False
[26382, 0]
False
[26383, 0]
False
[26384, 0]
False
[26385, 0]
False
[26386, 0]
False
[26387, 0]
False
[26388, 0]
False
[26389, 0]
False
[26390, 0]
False
[26391, 0]
False
[26392, 0]
False
[26393, 0]
False
[26394, 0]
False
[26395, 0]
False
[26396, 0]
False
[26397, 0]
False
[26398, 0]
False
[26399, 0]
False
[26400, 0]
False
[26401, 0]
False
[26402, 0]
False
[26403, 0]
False
[26404, 0]
False
[26405, 0]
False
[26406, 0]
False
[26407, 0]
False
[26408, 0]
False
[26409, 0]
False
[26410, 0]
False
[26411, 0]
False
[26412, 0]
False
[26413, 0]
False
[26414, 0]
False
[26415, 0]
False
[26416, 0]
False
[26417, 0]
False
[26418, 0]
False
[26419, 

False
[27361, 0]
False
[27362, 0]
False
[27363, 0]
False
[27364, 0]
False
[27365, 0]
False
[27366, 0]
False
[27367, 0]
False
[27368, 0]
False
[27369, 0]
False
[27370, 0]
False
[27371, 0]
False
[27372, 0]
False
[27373, 0]
False
[27374, 0]
False
[27375, 0]
False
[27376, 0]
False
[27377, 0]
False
[27378, 0]
False
[27379, 0]
False
[27380, 0]
False
[27381, 0]
False
[27382, 0]
False
[27383, 0]
False
[27384, 0]
False
[27385, 0]
False
[27386, 0]
False
[27387, 0]
False
[27388, 0]
False
[27389, 0]
False
[27390, 0]
False
[27391, 0]
False
[27392, 0]
False
[27393, 0]
False
[27394, 0]
False
[27395, 0]
False
[27396, 0]
False
[27397, 0]
False
[27398, 0]
False
[27399, 0]
False
[27400, 0]
False
[27401, 0]
False
[27402, 0]
False
[27403, 0]
False
[27404, 0]
False
[27405, 0]
False
[27406, 0]
False
[27407, 0]
False
[27408, 0]
False
[27409, 0]
False
[27410, 0]
False
[27411, 0]
False
[27412, 0]
False
[27413, 0]
False
[27414, 0]
False
[27415, 0]
False
[27416, 0]
False
[27417, 0]
False
[27418, 0]
False
[27419, 

[28110, 0]
False
[28111, 0]
False
[28112, 0]
False
[28113, 0]
False
[28114, 0]
False
[28115, 0]
False
[28116, 0]
False
[28117, 0]
False
[28118, 0]
False
[28119, 0]
False
[28120, 0]
False
[28121, 0]
False
[28122, 0]
False
[28123, 0]
False
[28124, 0]
False
[28125, 0]
False
[28126, 0]
False
[28127, 0]
False
[28128, 0]
False
[28129, 0]
False
[28130, 0]
False
[28131, 0]
False
[28132, 0]
False
[28133, 0]
False
[28134, 0]
False
[28135, 0]
False
[28136, 0]
False
[28137, 0]
False
[28138, 0]
False
[28139, 0]
False
[28140, 0]
False
[28141, 0]
False
[28142, 0]
False
[28143, 0]
False
[28144, 0]
False
[28145, 0]
False
[28146, 0]
False
[28147, 0]
False
[28148, 0]
False
[28149, 0]
False
[28150, 0]
False
[28151, 0]
False
[28152, 0]
False
[28153, 0]
False
[28154, 0]
False
[28155, 0]
False
[28156, 0]
False
[28157, 0]
False
[28158, 0]
False
[28159, 0]
False
[28160, 0]
False
[28161, 0]
False
[28162, 0]
False
[28163, 0]
False
[28164, 0]
False
[28165, 0]
False
[28166, 0]
False
[28167, 0]
False
[28168, 0]
Fal

KeyboardInterrupt: 

In [29]:
def ComputeGaps(gen):
    # Calculamos el cono.
    C = Cone(cone=gen)
    # Calculamos los rayos del cono.
    rayos = C.ExtremeRays()
    # Calculamos del diamante de multiplicidades.
    diamanteM = DiamondMultiplicity(rayos,gen)
    # Elegimos un punto en el interior del diamante.
    dimension = len(rayos[0])
    calibre  = [0 for x in range(dimension)]
    for ray in rayos:
        calibre = [calibre[i]+ray[i] for i in range(dimension)]
    x = []
    # Para cada rayo del cono.
    for ray in rayos:
        # Calculamos el conductor.
        conductor = ConductorAxis(gen,ray)
        # Calculamos las ecuaciones del rayo.
        eq = EqRay(ray,calibre)
        # Calculamos los términos afines.
        afinesDiamante = DeleteDuplicates(AffineTerm(eq,diamanteM))
        # Calculamos la máxima norma mínima en cada recta afin.
        maxMinNorm = 0
        for afin in afinesDiamante:
            # En primer lugar miramos la norma minima en el diamante.
            minimumNormDiamond = MinimumPointAffineDiamond(diamanteM,afin,eq)
            # Ahora buscamos el elemento mínimo en el diamante en dicha recta.
            minimumNormSG = ComputeMinimumNormInSemigroupLine(ray,minimumNormDiamond,gen)
            if maxMinNorm < minimumNormSG:
                maxMinNorm = minimumNormSG
        n = NormOne(conductor) + maxMinNorm
        print(n,ray,gen0)
        x.append(ComputeBoundRay(n,ray,gen0))
    return x

In [30]:
ComputeGaps(gen0)

7 [1, 0] [[4, 0], [3, 0], [5, 0], [5, 5], [2, 2], [3, 3], [4, 3], [5, 3], [5, 1], [6, 1], [7, 1], [5, 4]]


KeyboardInterrupt: 